![cover](./img/cover.png)
# Misión
En este ejercicio veremos como hacer algunos cálculos de trayectoria de propagación y cálculo de coeficiente de transmición y reflección utilizando la librería Pyrocko.
## Instalación
Para instalar Pyrocko sigue los siguientes pasos. Estas instrucciones funcionan bien en Mac y Linux, posiblemente requieras de algunos otros pasos adicionales si usas Windows. Las instrucciones para Windows las puedes encontrar en la página oficial de Pyrocko:

https://pyrocko.org/docs/current/install/system/windows.html

1. Crea un ambiente de conda para evitar conflictos con otros ambientes de python que tengas instalados. En esta configuración vamos a incluir el Jupyter para visualizar este cuaderno.
```bash
conda create -n pyrocko-env python=3.10
conda activate pyrocko-env
conda config --add channels conda-forge
conda install numpy scipy matplotlib obspy fftw proj geos
conda install -c pyrocko -c conda-forge pyrocko
conda install jupyter
python -m ipykernel install --user --name pyrocko-env --display-name "Pyrocko"
```
2. Para inicializar el cuaderno de Jupyter, usa la terminal para acceder al cuaderno donde vas a trabajar, y escribe.
```bash
jupyter notebook
```
Se abrirá una página de tu navegador dónde veras los archivos que se encuentran en ese directorio. Da click en el archivo llamado `Practica_01.ipynb`.

Puedes usar ChatGPT o DeepSeek para resolver dudas sobre la instalación.

En la siguiente celda simplemente vamos a cargar las librerias de Pyrocko. 

In [35]:
from pyrocko import cake
import numpy as np


Existe varios modelos de velocidad de la Tierra desarrollados por diferentes grupos. En la terminal los pudes listar con el comando `cake list-models`:
```bash
cake list-models
ak135-f-average-no-ocean.f
ak135-f-average-no-ocean.l
ak135-f-average-no-ocean.m
ak135-f-average.f
ak135-f-average.l
ak135-f-average.m
ak135-f-average.vf
ak135-f-continental.f
ak135-f-continental.l
ak135-f-continental.m
prem-no-ocean.f
prem-no-ocean.l
prem-no-ocean.m
prem.f
prem.l
prem.m
```
A continuación vamos a elegir uno de ellos para calcular los arribos de ondas P y S. 

In [37]:
km = 1000.

# Load builtin 'prem-no-ocean' model (medium resolution)
model = cake.load_model('prem-no-ocean.m')


A continuación vamos a definir algunas fases. Por ejemplo, en la documentación puedes entrar una fase como esta

``` bash
y_crazy_phase = PhaseDef('pPv(moho)sP\\')

print my_crazy_phase
Phase definition "pPv(moho)sP":
 - P mode propagation, departing upward
 - surface reflection
 - P mode propagation, departing downward
 - upperside reflection with conversion from P to S at moho
 - S mode propagation, departing upward
 - surface reflection with conversion from S to P
 - P mode propagation, departing downward
 - arriving at target from above```
 
 Sin embargo, empecemos con algo sencillo, como una onda P o S. 
 
 En la siguiente celda puedes intentar diferentes casos:

In [53]:
Phase = cake.PhaseDef('P')
print(Phase)


Phase definition "P":
 - P mode propagation, departing downward
 - arriving at target from below


En la siguiente celda vamos a calcular los tiempos de arribo. 

In [63]:
# Profundidad de la fuente [m].
source_depth = 30. * km

# Distancias as a numpy array [deg].
# num.linspace(distancia_inicial, distancia_final, no. de puntos)
distances = num.linspace(200, 500, 5)*km * cake.m2d


# calculate distances and arrivals and print them:
print('distancia [km]   tiempo [s]')
for arrival in model.arrivals(distances, phases=Phase, zstart=source_depth):
    print('%10.2f     %10.2f' % (arrival.x*cake.d2m/km, arrival.t))

distancia [km]   tiempo [s]
    425.00          54.76
    500.00          63.96


# Pregunta 1. 
Mediante prueba a error, calcula a que distancia en kilómetrods llegaría una onda P y una S despúes de **15 minutos** si la fuente esta localizada a **20km** de profundidad. Antes de intentar resolver el ejercicio escribe en la siguiente celda cuanto kilómetros crees que recorre una onda **P** en **15 minutos. Nota: No hay respuesta correcta o incorrecta.**


Tiempo de viaje de la onda P (estimado):_______________ segundos 

Despúes de haber realizado el cálculo escribe cuales son los tiempos de viaje que obtuviste para la **onda P** y la **onda S**, escribe entre parentesis el porcentaje de la circunferencia de la Tierra que recorrió, 50% equivale a 1/2 vuelta al mundo:

Tiempo de viaje de la onda P (calculado): _________ segundos, (____)% circunferencia de la Tierra.

Tiempo de viaje de la onda S (calculado): _________ segundos, (____)% circunferencia de la Tierra.

# Matriz de esparcimiento
Como vimos en clase la matriz de espacimiento (*scattering matriz*) permite obtener los coeficientes de reflexión y refracción en la interfaz entre capas. Para calcularla primero debemos de definir un material. Para ellos vamos a utilizar las subrutinas `cake.Material`. Si corres la siguiente línea sin argumentos obtendrás los valores típicos de la corteza:
```python
cake.Material()
Material(vp=5800.0, vs=3348.6315612998296, rho=2600.0, qp=1349.9999999999998, qs=600.0)
```
Intenta cambiar alguno de estos parámetros a la vez y verás como se actualiza. A menos que determines un valor de coeficiente de Poisson diferente de 0.5, el programa asume que se trata de un medio de Poisson para calcular los demás valores. Por ejemplo, cambia la velocidad de la **onda S** a **10km/s**,
```python
cake.Material(vs=10000)
```
en la siguiente celda. Confirma que la tasa entre la **onda S** y la **onda P**, es igual a sqrt(3). Te suguiero intentar cambiado otros valores. En un medio isotrópico únicamente existen dos parámetros elásticos independientes.

In [65]:
# Escribe tu código aquí:

Para calcular los matriz de espacimiento para el caso de una onda P-SV vamos a suponer que el medio superior tiene los valores por default para la corteza, y que el medio inferior tiene una velocidad de **onda P**, **20% mayor** que el medio superior. Calcula la matriz de esparcimiento usando las funciones:
```python
cake.Material()
cake.psv_solid()
```
La documentación de estas funciones las puedes encontrar aquí:
https://pyrocko.org/docs/current/library/reference/pyrocko.cake.html#pyrocko.cake.PhaseDef

In [ ]:
# Escribe tu código aquí

Pyrocko te permite visualizar fácilmente los trayectorias de propagación utilizando CLI (Command Line Interphase). Los comando a continuación no corren dentro de este cuaderno de Jupyter, correlos directamente en la terminal:
``` bash
cake plot-rays --crust2loc=45,10 --phases=P,p --sdepth=10 --distances=100:1000:10
```
Al correr este comando obtendras las trayectorias para el modelo de velocidades Curst2.0 para las **ondas P y p** para una fuente localizada a una latitud de **45 grados** y una **longitud de 10 grados** a **10km de profundidad**. En este caso vamos a colocar **10 receptores (sensores)** entre **100 y 1000 km** de distancia. Al correrlo obtendrás la siguiente gráfica. 
![Cake model](./img/cake_model.png)

# Ejercicio 1
Calcula las trayectorias de propagacción para los siguientes casos. 

1. Para las ondas P y S para un sismo que ocurre en medio del oceáno Pacífico. 
2. Las ondas p,P y s,S para un sismo que ocurre en cratón (ejemplo, Brasil, Canada, Rusia).
3. Para un sismos en un lugar que tu eligas (indicalo en tu respuesta) con profundidades de 10 y 100 km. 

Para incluir tus imágenes en el cuaderno de Jupyter guardalas con el nombre que prefieras (ejemplo, *figura1.png, figura2.png, figura3.png*) y ultiliza el siguiente comando en la próxima celda. 

```markdown
![caption](figura1.png)
![caption](figura2.png)
![caption](figura3.png)
```
Puedes sustituir la palabra caption por cualquier otro texto.